## Assignment 2
* Implement a linear chain CRF with a library
* Use it on two data sets with the same set of features
* Implement the model and evaluate with F-Score. How are the feature weights which are learnt different between the models?

## File handling and data loading
Loads data from files, for both POS training&testing and NER training&testing files

In [1]:
"""
Notebook Imports
"""
import os
import pycrfsuite # pip install python-crfsuite
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [2]:
class DataLoader:
    """
    Loads input data files and cleans up data into following format:
    
    eg: (Data1, Type1)
    """
    #
    def __init__(self, POS_train_path, POS_test_path, NER_train_path, NER_test_path):
        """
        Loads data files into memory
        """
        __POS_train_path = POS_train_path
        __POS_test_path = POS_test_path
        __NER_train_path = NER_train_path
        __NER_test_path = NER_test_path
        #
        # Loads data from disk into memory
        __POS_train_data, __POS_test_data, __NER_train_data, __NER_test_data = self.__extract_data_from_files(POS_train_path=__POS_train_path,
                                                                                                              POS_test_path=__POS_test_path,
                                                                                                              NER_train_path=__NER_train_path,
                                                                                                              NER_test_path=__NER_test_path)
        #
        # Formats data currently loaded in memory into a formatted structure
        self.__POS_train_structured_data, self.__POS_test_structured_data, self.__NER_train_structured_data,self.__NER_test_structured_data=self.__format_data(POS_train_data=__POS_train_data,
                                                                                                                                                               POS_test_data=__POS_test_data,
                                                                                                                                                               NER_train_data=__NER_train_data,
                                                                                                                                                               NER_test_data=__NER_test_data)
    #
    def __extract_data_from_files(self, POS_train_path, POS_test_path, NER_train_path, NER_test_path):
        """
        Opens data files and returns data in memory
        """
        #
        BASE_DIR = os.path.join( os.path.dirname(os.getcwd()))
        #
        with open(BASE_DIR + POS_train_path) as f:
            POS_train_data = f.read()
        with open(BASE_DIR + POS_test_path) as f:
            POS_test_data = f.read()
        with open(BASE_DIR + NER_train_path) as f:
            NER_train_data = f.read()
        with open(BASE_DIR + NER_test_path) as f:
            NER_test_data = f.read()
        #
        return POS_train_data, POS_test_data, NER_train_data, NER_test_data
    #
    def __format_data(self, POS_train_data, POS_test_data, NER_train_data, NER_test_data):
        """
        Formats data into a python data structure (list of lists)
        eg: (Data1, Type1)
        """
        POS_train_structured_data, POS_test_structured_data, NER_train_structured_data, NER_test_structured_data = [],[],[],[]
        temp_list=[]
        for line in POS_train_data.split("\n"):
            if line is not None and line != "": 
                sub_list = line.split("\t")
                temp_list.append(sub_list)
            else:
                POS_train_structured_data.append(temp_list)
                temp_list = []
        temp_list = []
        for line in POS_test_data.split("\n"):
            if line is not None and line != "": 
                sub_list = line.split("\t")
                temp_list.append(sub_list)
            else:
                POS_test_structured_data.append(temp_list)
                temp_list = []
        temp_list = []
        for line in NER_train_data.split("\n"):
            if line is not None and line != "": 
                sub_list = line.split("\t|")
                temp_list.append(sub_list)
            else:
                NER_train_structured_data.append(temp_list)
                temp_list = []
        #
        temp_list = []
        for line in NER_test_data.split("\n"):
            if line is not None and line != "": 
                sub_list = line.split("\t|")
                temp_list.append(sub_list)
            else:
                NER_test_structured_data.append(temp_list)
                temp_list = []
        #
        return POS_train_structured_data, POS_test_structured_data, NER_train_structured_data, NER_test_structured_data
    #
    def load_data(self):
        """
        Method wrapper which loads data into memory and returns all relevant training and testing files
        """
        return self.__POS_train_structured_data, self.__POS_test_structured_data, self.__NER_train_structured_data, self.__NER_test_structured_data
#
data_loader_obj = DataLoader(POS_train_path="\\data\\pos\\train.col",
                             POS_test_path="\\data\\pos\\test.col",
                             NER_train_path="\\data\\ner-pol\\train.iob",
                             NER_test_path="\\data\\ner-pol\\test.iob")
POS_train_data, POS_test_data, NER_train_data, NER_test_data = data_loader_obj.load_data()
print("Example: " + str(POS_train_data[0:5]))
print("------------------------------------")
print("Example: " + str(POS_test_data[0:5]))
print("------------------------------------")
print("Example: " + str(NER_train_data[0:5]))
print("------------------------------------")
print("Example: " + str(NER_test_data[0:5]))

Example: [[['In', 'IN'], ['an', 'DT'], ['Oct.', 'NNP'], ['19', 'CD'], ['review', 'NN'], ['of', 'IN'], ['``', '``'], ['The', 'DT'], ['Misanthrope', 'NN'], ["''", "''"], ['at', 'IN'], ['Chicago', 'NNP'], ["'s", 'POS'], ['Goodman', 'NNP'], ['Theatre', 'NNP'], ['(', '-LRB-'], ['``', '``'], ['Revitalized', 'VBN'], ['Classics', 'NNS'], ['Take', 'VBP'], ['the', 'DT'], ['Stage', 'NN'], ['in', 'IN'], ['Windy', 'NNP'], ['City', 'NNP'], [',', ','], ["''", "''"], ['Leisure', 'NN'], ['&', 'CC'], ['Arts', 'NNS'], [')', '-RRB-'], [',', ','], ['the', 'DT'], ['role', 'NN'], ['of', 'IN'], ['Celimene', 'NNP'], [',', ','], ['played', 'VBN'], ['by', 'IN'], ['Kim', 'NNP'], ['Cattrall', 'NNP'], [',', ','], ['was', 'VBD'], ['mistakenly', 'RB'], ['attributed', 'VBN'], ['to', 'IN'], ['Christina', 'NNP'], ['Haag', 'NNP'], ['.', '.']], [['Ms.', 'NNP'], ['Haag', 'NNP'], ['plays', 'VBZ'], ['Elianti', 'NNP'], ['.', '.']], [['Rolls', 'NNP'], ['-', 'HYPH'], ['Royce', 'NNP'], ['Motor', 'NNP'], ['Cars', 'NNPS'], ['Inc.'

## Feature Enhancer

Adds more features to the already loaded vectors, including the following features:

* The word itself, coverted to lower case
* Word Suffix (-2,-3)
* Boolean if string is uppercased
* Boolean if string is a title (eg: Title)
* Boolean if string is a digit
* The postag
* The string before it, converted to lowercase
* Boolean if the string before it is a title (eg: Title)
* Boolean if the string before it is uppercased
* Boolean if the string before it is a digit
* postag of string before it
* The string after it, converted to lowercase
* Boolean if the string after it is a title (eg: Title)
* Boolean if the string after it is uppercased
* Boolean if the string after it is a digit
* postag of string after it

In [3]:
def word2features(doc, i, model_type):
    """
    Accepts a word, and respective POS tag, and converts it into a vector of features
    
    Applies different features to the model, depending on whether we are tackling POS tagging, or NER tagging.
    """
    #print(doc[i])
    word = doc[i][0]
    postag = doc[i][1]
    #
    if model_type == 0: # POS Tagging Features
        #
        features = [
            'bias',
            'word.lower=' + word.lower(),
            'word[-3:]=' + word[-3:],
            'word[-2:]=' + word[-2:],
            'word.isupper=%s' % word.isupper(),
            'word.istitle=%s' % word.istitle(),
            'word.isdigit=%s' % word.isdigit(),
            'postag=' + postag
        ]
        #
        # Preword
        if i > 0:
            word1 = doc[i-1][0]
            postag1 = doc[i-1][1]
            features.extend([
                '-1:word.lower=' + word1.lower(),
                '-1:word.istitle=%s' % word1.istitle(),
                '-1:word.isupper=%s' % word1.isupper(),
                '-1:word.isdigit=%s' % word1.isdigit(),
                '-1:postag=' + postag1
            ])
        else:
            # Beginning of document
            features.append('BOS')
        # 
        # Postword
        if i < len(doc)-1:
            word1 = doc[i+1][0]
            postag1 = doc[i+1][1]
            features.extend([
                '+1:word.lower=' + word1.lower(),
                '+1:word.istitle=%s' % word1.istitle(),
                '+1:word.isupper=%s' % word1.isupper(),
                '+1:word.isdigit=%s' % word1.isdigit(),
                '+1:postag=' + postag1
            ])
        else:
            # End of document
            features.append('EOS')
    else:
        #
        features = [
            'bias',
            'word.lower=' + word.lower(),
            'word[-3:]=' + word[-3:],
            'word[-2:]=' + word[-2:],
            'word.isupper=%s' % word.isupper(),
            'word.istitle=%s' % word.istitle(),
            'word.isdigit=%s' % word.isdigit(),
            'postag=' + postag
        ]
        #
        # Preword
        if i > 0:
            word1 = doc[i-1][0]
            postag1 = doc[i-1][1]
            features.extend([
                '-1:word.lower=' + word1.lower(),
                '-1:word.istitle=%s' % word1.istitle(),
                '-1:word.isupper=%s' % word1.isupper(),
                '-1:word.isdigit=%s' % word1.isdigit(),
                '-1:postag=' + postag1
            ])
        else:
            # Beginning of document
            features.append('BOS')
        #
        # Postword
        if i < len(doc)-1:
            word1 = doc[i+1][0]
            postag1 = doc[i+1][1]
            features.extend([
                '+1:word.lower=' + word1.lower(),
                '+1:word.istitle=%s' % word1.istitle(),
                '+1:word.isupper=%s' % word1.isupper(),
                '+1:word.isdigit=%s' % word1.isdigit(),
                '+1:postag=' + postag1
            ])
        else:
            # End of document
            features.append('EOS')
    #    
    return features
#
# A function for extracting features in documents
def extract_features(doc, model_type):
    return [word2features(doc, i, model_type) for i in range(len(doc))]
#
# A function for generating the list of labels for each document
def get_labels(doc):
    return [label for (token, label) in doc]
#
# A function for generating the list of features for each document
def get_features(doc):
    return [token for (token, label) in doc]
#
print(POS_train_data[:5])
print(NER_train_data[:5])
print("------------------------------------")
X_POS_train_data = [extract_features(doc, 0) for doc in POS_train_data]
y_POS_train_data = [get_labels(doc) for doc in POS_train_data]
X_NER_train_data = [extract_features(doc, 1) for doc in NER_train_data]
y_NER_train_data = [get_labels(doc) for doc in NER_train_data]
print("------------------------------------")
print("POS Train Data Snippet:")
print(X_POS_train_data[:5])
print(y_POS_train_data[:5])
print("------------------------------------")
print("NER Train Data Snippet:")
print(X_NER_train_data[:5])
print(y_NER_train_data[:5])

[[['In', 'IN'], ['an', 'DT'], ['Oct.', 'NNP'], ['19', 'CD'], ['review', 'NN'], ['of', 'IN'], ['``', '``'], ['The', 'DT'], ['Misanthrope', 'NN'], ["''", "''"], ['at', 'IN'], ['Chicago', 'NNP'], ["'s", 'POS'], ['Goodman', 'NNP'], ['Theatre', 'NNP'], ['(', '-LRB-'], ['``', '``'], ['Revitalized', 'VBN'], ['Classics', 'NNS'], ['Take', 'VBP'], ['the', 'DT'], ['Stage', 'NN'], ['in', 'IN'], ['Windy', 'NNP'], ['City', 'NNP'], [',', ','], ["''", "''"], ['Leisure', 'NN'], ['&', 'CC'], ['Arts', 'NNS'], [')', '-RRB-'], [',', ','], ['the', 'DT'], ['role', 'NN'], ['of', 'IN'], ['Celimene', 'NNP'], [',', ','], ['played', 'VBN'], ['by', 'IN'], ['Kim', 'NNP'], ['Cattrall', 'NNP'], [',', ','], ['was', 'VBD'], ['mistakenly', 'RB'], ['attributed', 'VBN'], ['to', 'IN'], ['Christina', 'NNP'], ['Haag', 'NNP'], ['.', '.']], [['Ms.', 'NNP'], ['Haag', 'NNP'], ['plays', 'VBZ'], ['Elianti', 'NNP'], ['.', '.']], [['Rolls', 'NNP'], ['-', 'HYPH'], ['Royce', 'NNP'], ['Motor', 'NNP'], ['Cars', 'NNPS'], ['Inc.', 'NNP'],

## Training Model - POS

In [4]:
file_path = "POS_crf.model"
file_exists = os.path.exists(file_path)
if(not file_exists):
    trainer_POS = pycrfsuite.Trainer(verbose=True)
    #
    # Submit training data to the trainer
    print('Submitting POS dataset..')
    for xseq, yseq in zip(X_POS_train_data, y_POS_train_data):
        trainer_POS.append(xseq,yseq)
    #
    # Set the parameters of the model
    trainer_POS.set_params({
        # coefficient for L1 penalty
        'c1': 0.1,

        # coefficient for L2 penalty
        'c2': 0.01,  

        # maximum number of iterations
        'max_iterations': 50,

        # whether to include transitions that
        # are possible, but not observed
        'feature.possible_transitions': True
    })
    #
    # Provide a file name as a parameter to the train function, such that
    # the model will be saved to the file when training is finished
    print('Training POS dataset..')
    trainer_POS.train(file_path)
    print('POS Model Trained..')

Submitting POS dataset..
Training POS dataset..
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 258754
Seconds required: 4.699

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 2356861.799877
Feature norm: 1.000000
Error norm: 389760.910793
Active features: 255541
Line search trials: 1
Line search step: 0.000002
Seconds required for this iteration: 16.934

***** Iteration #2 *****
Loss: 1762557.976667
Feature norm: 4.204090
Error norm: 428090.323335
Active features: 247704
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 8.483

***** Iteration #3 *****
Loss: 1466185.176201
Feature norm: 5.616498
Error norm: 338744.516722
Active features: 252348
Line sea

## Training Model - NER

In [5]:
file_path = "NER_crf.model"
file_exists = os.path.exists(file_path)
if(not file_exists):
    trainer_NER = pycrfsuite.Trainer(verbose=True)
    #
    # Submit training data to the trainer
    print('Submitting NER dataset..')
    for xseq, yseq in zip(X_NER_train_data, y_NER_train_data):
        trainer_NER.append(xseq, yseq)
    #
    # Set the parameters of the model
    trainer_NER.set_params({
        # coefficient for L1 penalty
        'c1': 0.1,

        # coefficient for L2 penalty
        'c2': 0.01,  

        # maximum number of iterations
        'max_iterations': 50,

        # whether to include transitions that
        # are possible, but not observed
        'feature.possible_transitions': True
    })
    #
    # Provide a file name as a parameter to the train function, such that
    # the model will be saved to the file when training is finished   
    print('Training NER dataset..')
    trainer_NER.train(file_path)
    print('NER Model Trained..')

Submitting NER dataset..
Training NER dataset..
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 85604
Seconds required: 1.003

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 145292.820743
Feature norm: 1.000000
Error norm: 56950.043777
Active features: 85301
Line search trials: 1
Line search step: 0.000002
Seconds required for this iteration: 0.685

***** Iteration #2 *****
Loss: 139727.165841
Feature norm: 1.086470
Error norm: 46209.750236
Active features: 81832
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.344

***** Iteration #3 *****
Loss: 122947.894177
Feature norm: 1.365305
Error norm: 49167.815793
Active features: 81441
Line search trials:

## Running models on Test Data

In [6]:
X_POS_test_data = [extract_features(doc, 0) for doc in POS_test_data]
y_POS_test_data = [get_labels(doc) for doc in POS_test_data]
X_NER_test_data = [extract_features(doc, 1) for doc in NER_test_data]
y_NER_test_data = [get_labels(doc) for doc in NER_test_data]
print("POS Train Data Snippet:")
print(X_POS_test_data[:5])
print(y_POS_test_data[:5])
print("NER Train Data Snippet:")
print(X_NER_test_data[:5])
print(y_NER_test_data[:5])
print("--------------------")
#
# Running test data on POS model   
tagger_POS = pycrfsuite.Tagger()
tagger_POS.open('POS_crf.model')
y_POS_pred = [tagger_POS.tag(xseq) for xseq in X_POS_test_data]
#
# Running test data on NER model   
tagger_NER = pycrfsuite.Tagger()
tagger_NER.open('NER_crf.model')
y_NER_pred = [tagger_NER.tag(xseq) for xseq in X_NER_test_data]
#
print("POS Pred Data Snippet:")
print(y_POS_pred[:5])
print("--------------------")
print("NER Pred Data Snippet:")
print(y_NER_pred[:5])

POS Train Data Snippet:
[[['bias', 'word.lower=measuring', 'word[-3:]=ing', 'word[-2:]=ng', 'word.isupper=False', 'word.istitle=True', 'word.isdigit=False', 'postag=NN', 'BOS', '+1:word.lower=cups', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=NNS'], ['bias', 'word.lower=cups', 'word[-3:]=ups', 'word[-2:]=ps', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=NNS', '-1:word.lower=measuring', '-1:word.istitle=True', '-1:word.isupper=False', '-1:word.isdigit=False', '-1:postag=NN', '+1:word.lower=may', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=MD'], ['bias', 'word.lower=may', 'word[-3:]=may', 'word[-2:]=ay', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=MD', '-1:word.lower=cups', '-1:word.istitle=False', '-1:word.isupper=False', '-1:word.isdigit=False', '-1:postag=NNS', '+1:word.lower=soon', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isd

POS Pred Data Snippet:
[['NN', 'NNS', 'MD', 'RB', 'VB', 'VBN', 'IN', 'NNS', 'IN', 'DT', 'NN', 'NN', '.'], ['NNP', 'CC', 'NNP', 'NNP', 'VBZ', 'TO', 'VB', 'VBG', 'JJ', 'NN', 'DT', 'JJ', 'NN', 'WDT', 'MD', 'VB', 'RB', 'DT', 'JJ', 'NNS', 'IN', 'NN', '.'], ['DT', 'NN', 'VBZ', 'IN', 'NNS', 'VBN', 'IN', 'NNP', 'WRB', 'JJ', 'NNS', 'VBP', 'VBD', 'JJ', 'NN', 'IN', 'JJ', 'NNS', '.'], ['PRP', 'RB', 'VBZ', 'NNP', 'POS', 'VBG', 'NN', 'IN', 'PRP$', 'JJ', 'NNS', ',', 'JJ', 'IN', 'NNP', 'NNP', ',', 'MD', 'VB', 'PRP$', 'NNS', 'IN', 'DT', 'NNP', '.'], ['DT', 'NNP', 'NN', 'HYPH', 'NNS', 'NN', 'VBD', 'VBN', 'CD', 'NNS', 'RB', 'IN', 'NNP', 'WRB', 'NNP', 'VBD', 'DT', 'JJ', 'NN', ',', 'VBN', 'NNP', ',', 'WDT', 'RB', 'VBD', 'DT', 'CD', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NNS', '.']]
--------------------
NER Pred Data Snippet:
[['O'], ['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O'], ['B-PER', 'I-PER'], ['B-LOC', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O'], ['B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', '

## Evaluation

In [7]:
average='weighted'
#
# Flatten Lists
y_POS_test_data_flat, y_POS_pred_flat, y_NER_test_data_flat, y_NER_pred_flat = [],[],[],[]
for sub_list in y_POS_test_data:
    for item in sub_list:
        y_POS_test_data_flat.append(item)
for sub_list in y_POS_pred:
    for item in sub_list:
        y_POS_pred_flat.append(item)
for sub_list in y_NER_test_data:
    for item in sub_list:
        y_NER_test_data_flat.append(item)
for sub_list in y_NER_pred:
    for item in sub_list:
        y_NER_pred_flat.append(item)
print(y_POS_test_data_flat[:30])
print(y_POS_pred_flat[:30])
print(y_NER_test_data_flat[:30])
print(y_NER_pred_flat[:30])
#
# POS Evaluation
accuracy = accuracy_score(y_POS_test_data_flat, y_POS_pred_flat)
precision = precision_score(y_POS_test_data_flat, y_POS_pred_flat, average=average)
recall = recall_score(y_POS_test_data_flat, y_POS_pred_flat, average=average)
f1_s = f1_score(y_POS_test_data_flat, y_POS_pred_flat, average=average) 
print("POS Accuracy: " + str(accuracy))
print("POS Precision: " + str(precision))
print("POS Recall: " + str(recall))
print("POS F1 Score: " + str(f1_s) + "\n")
#
# NER Evaluation
accuracy = accuracy_score(y_NER_test_data_flat, y_NER_pred_flat)
precision = precision_score(y_NER_test_data_flat, y_NER_pred_flat, average=average)
recall = recall_score(y_NER_test_data_flat, y_NER_pred_flat, average=average)
f1_s = f1_score(y_NER_test_data_flat, y_NER_pred_flat, average=average)
print("NER Accuracy: " + str(accuracy))
print("NER Precision: " + str(precision))
print("NER Recall: " + str(recall))
print("NER F1 Score: " + str(f1_s))
#
# Print out the classification report
print("POS Classification Report")
print(classification_report(
    y_POS_test_data_flat, y_POS_pred_flat,
    target_names=["NN", "NNS","MD","RB","VB","VBN","IN","DT","NNP","VBZ","TO","VBG","JJ","WDT"]))
print("NER Classification Report")
print(classification_report(
    y_NER_test_data_flat, y_NER_pred_flat,
    target_names=["B-LOC", "O","I-MISC","I-LOC","B-PER","B-MISC"]))

['NN', 'NNS', 'MD', 'RB', 'VB', 'VBN', 'IN', 'NNS', 'IN', 'DT', 'NN', 'NN', '.', 'NNP', 'CC', 'NNP', 'NNP', 'VBZ', 'TO', 'VB', 'VBG', 'JJ', 'NN', 'DT', 'JJ', 'NN', 'WDT', 'MD', 'VB', 'RB']
['NN', 'NNS', 'MD', 'RB', 'VB', 'VBN', 'IN', 'NNS', 'IN', 'DT', 'NN', 'NN', '.', 'NNP', 'CC', 'NNP', 'NNP', 'VBZ', 'TO', 'VB', 'VBG', 'JJ', 'NN', 'DT', 'JJ', 'NN', 'WDT', 'MD', 'VB', 'RB']
['O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'B-LOC', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'O']
['O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'B-LOC', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'O']
POS Accuracy: 1.0
POS Precision: 1.0
POS Recall: 1.0
POS F1 Score: 1.0

NER Accuracy: 1.0
NER Precision: 1.0
NER Recall: 1.0
NER F1 Score: 1.0
POS Classification Report
             precision    recall  f1-score   support

     

c:\python36\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 45, does not match size of target_names, 14
  .format(len(labels), len(target_names))
c:\python36\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 9, does not match size of target_names, 6
  .format(len(labels), len(target_names))
